In [12]:
import pandas as pd
import numpy as np
import spacy

#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embedding
import gensim
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [13]:
filepath = "tranlated_tweets.csv"

In [19]:
df = pd.read_csv(filepath)
nlp = spacy.load('en_core_web_sm')

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 4 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Description                     2000 non-null   object
 1   Analysis results                2000 non-null   int64 
 2   Description Translated          2000 non-null   object
 3   Description Cleaned Translated  2000 non-null   object
dtypes: int64(1), object(3)
memory usage: 62.6+ KB


In [21]:
import nltk
nltk.download('all')

# create a list text
text = list(df['Description Cleaned Translated'])


# preprocessing loop
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
corpus = []

for i in range(len(text)):
    r = re.sub('[^a-zA-Z]', ' ', text[i])
    r = r.lower()
    r = r.split()
    r = [word for word in r if word not in stopwords.words('english')]
    r = [lemmatizer.lemmatize(word) for word in r]
    r = ' '.join(r)
    corpus.append(r)

#assign corpus to data['text']
df['lemma'] = corpus
df.head()

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/n

,Description,Analysis results,Description Translated,Description Cleaned Translated,lemma
0,@liznorthon @SoyAIvaro @ursulaeggers @MielyMer...,1,@liznorthon @SoyAIvaro @ursulaeggers @MielyMer...,"They caught the suspect, he is a Colombian nat...",caught suspect colombian national surprise vio...
1,@ccarvajaly Cuando se den cuenta que eran homi...,1,@ccarvajaly When they realize that they were m...,When they realize that they were murderers or ...,realize murderer drug trafficker caught say fo...
2,@SergioChouza @santidemoreno11 Jajaajja ahí es...,1,@SergioChouza @santidemoreno11 Hahaha you were...,"Hahaha, you were good there, you can see YOUR ...",hahaha good see sudaca head bad payer dna piec...
3,"Acá podemos ver a un alemán, de cabello rubio,...",1,"Here we can see a German, with blonde hair, bl...","Here we can see a German, with blonde hair, bl...",see german blonde hair blue eye beating shitty...
4,@Senado_Chile Murieron 8 compatriotas por culp...,1,@Senado_Chile 8 compatriots died because of an...,8 compatriots died because of an immigrant who...,compatriot died immigrant never set foot chile...


In [22]:
#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower()
    text=text.strip()
    text=re.compile('<.*?>').sub('', text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)
    text = re.sub('\s+', ' ', text)
    text = re.sub(r'\[[0-9]*\]',' ',text)
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text)
    text = re.sub(r'\s+',' ',text)
    return text


# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)
#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()

# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))

In [23]:
df['lemma_v2'] = df['Description Cleaned Translated'].apply(lambda x: finalpreprocess(x))
df.head()

,Description,Analysis results,Description Translated,Description Cleaned Translated,lemma,lemma_v2
0,@liznorthon @SoyAIvaro @ursulaeggers @MielyMer...,1,@liznorthon @SoyAIvaro @ursulaeggers @MielyMer...,"They caught the suspect, he is a Colombian nat...",caught suspect colombian national surprise vio...,caught suspect colombian national surprise vio...
1,@ccarvajaly Cuando se den cuenta que eran homi...,1,@ccarvajaly When they realize that they were m...,When they realize that they were murderers or ...,realize murderer drug trafficker caught say fo...,realize murderer drug trafficker catch say fin...
2,@SergioChouza @santidemoreno11 Jajaajja ahí es...,1,@SergioChouza @santidemoreno11 Hahaha you were...,"Hahaha, you were good there, you can see YOUR ...",hahaha good see sudaca head bad payer dna piec...,hahaha good see sudaca head bad payer dna piec...
3,"Acá podemos ver a un alemán, de cabello rubio,...",1,"Here we can see a German, with blonde hair, bl...","Here we can see a German, with blonde hair, bl...",see german blonde hair blue eye beating shitty...,see german blonde hair blue eye beat shitty bl...
4,@Senado_Chile Murieron 8 compatriotas por culp...,1,@Senado_Chile 8 compatriots died because of an...,8 compatriots died because of an immigrant who...,compatriot died immigrant never set foot chile...,compatriot die immigrant never set foot chile ...


## modelling

### BoG and TF-IDF

In [24]:
X = df['lemma']
y = df['Analysis results']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(use_idf=True)
cv = CountVectorizer()

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_train_cv = cv.fit_transform(X_train)

In [10]:
# Training Logistic Regression model

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
lr = LogisticRegression()
lr.fit(X_train_tfidf, y_train)

forest = RandomForestClassifier()
forest.fit(X_train_tfidf, y_train)

# transform X_test using CV
X_test_tfidf = tfidf_vectorizer.transform(X_test)


# generate predictions
predictions = lr.predict(X_test_tfidf)
predictions = forest.predict(X_test_tfidf)

In [11]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [12]:
accuracy_score(y_test, predictions)

0.7475

In [13]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.73      0.78      0.75       198
           1       0.77      0.71      0.74       202

    accuracy                           0.75       400
   macro avg       0.75      0.75      0.75       400
weighted avg       0.75      0.75      0.75       400



### Deep learning

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
from collections import defaultdict

In [25]:
X = df['lemma']
y = df['Analysis results']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [26]:
BATCH_SIZE = 64
VOCAB_SIZE = 6000
LSTM_LAYER = 30
HIDDEN_DIM = 100
OUTPUT_DIM = 257

In [27]:
tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(X)

def text_to_seq(text):
    sequences = tokenizer.texts_to_sequences(text)
    return sequence.pad_sequences(sequences, maxlen=40)

In [36]:
class TweetClassifier(nn.ModuleList):

	def __init__(self):
		super(TweetClassifier, self).__init__()

		self.batch_size = 64
		self.hidden_dim = 100
		self.LSTM_layers = 20
		self.input_size = 6000

		self.dropout = nn.Dropout(0.5)
		self.embedding = nn.Embedding(self.input_size, self.hidden_dim, padding_idx=0)
		self.lstm = nn.LSTM(input_size=self.hidden_dim, hidden_size=self.hidden_dim, num_layers=self.LSTM_layers, batch_first=True)
		self.fc1 = nn.Linear(in_features=self.hidden_dim, out_features=257)
		self.fc2 = nn.Linear(257, 1)

	def forward(self, x):

		h = torch.zeros((self.LSTM_layers, x.size(0), self.hidden_dim))
		c = torch.zeros((self.LSTM_layers, x.size(0), self.hidden_dim))

		torch.nn.init.xavier_normal_(h)
		torch.nn.init.xavier_normal_(c)

		out = self.embedding(x)
		out, (hidden, cell) = self.lstm(out, (h,c))
		out = self.dropout(out)
		out = torch.relu_(self.fc1(out[:,-1,:]))
		out = self.dropout(out)
		out = torch.sigmoid(self.fc2(out))

		return out

In [37]:
class RacismDataset(Dataset):
    def __init__(self, x, y):
        self.seq_tweets = text_to_seq(x)
        self.labels = y

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        tweet = self.seq_tweets[idx] #numpy
        label = self.labels.iloc[idx] #pandas df

        return tweet, label

In [38]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(RacismDataset(X_train, y_train), batch_size=64, shuffle=True)
test_dataloader = DataLoader(RacismDataset(X_test, y_test))

In [39]:
model = TweetClassifier()
optimizer = optim.RMSprop(model.parameters(), lr=1e-4)

In [40]:
class MetricMonitor:
    def __init__(self, float_precision=3):
        self.float_precision = float_precision
        self.reset()

    def reset(self):
        self.metrics = defaultdict(lambda: {"val": 0, "count": 0, "avg": 0})

    def update(self, metric_name, val):
        metric = self.metrics[metric_name]

        metric["val"] += val
        metric["count"] += 1
        metric["avg"] = metric["val"] / metric["count"]

    def __str__(self):
        return " | ".join(
            [
                "{metric_name}: {avg:.{float_precision}f}".format(
                    metric_name=metric_name, avg=metric["avg"], float_precision=self.float_precision
                )
                for (metric_name, metric) in self.metrics.items()
            ]
        )

In [41]:
def train(model, train_loader, optimizer, epoch, device):
    model = model.to(device)
    model.train()
    train_loss = 0

    monitor = MetricMonitor()
    # loss_function = CrossEntropyLoss()

    stream = tqdm(train_loader)
    for data, target in stream:
        data = data.to(device)
        target = target.to(device)

        data = data.type(torch.LongTensor)
        target = target.type(torch.FloatTensor)

        optimizer.zero_grad()
        output = model(data).squeeze(1)
        loss = F.binary_cross_entropy(output, target)
        train_loss += loss.sum().item()
        loss.backward()
        optimizer.step()

        # logs
        monitor.update("loss", loss.item())
        stream.set_description(
            "Epoch: {epoch}. {monitor}".format(epoch=epoch, monitor=monitor)
        )

    return monitor


def test(model, test_loader, epoch, device):
    model = model.to(device)
    model.eval()

    monitor = MetricMonitor()
    # loss_function = CrossEntropyLoss()

    stream = tqdm(test_loader)
    with torch.no_grad():
        for data, target in stream:
            data = data.to(device)
            target = target.to(device)

            data = data.type(torch.LongTensor)
            target = target.type(torch.FloatTensor)

            output = model(data).squeeze(1)

            test_loss = F.binary_cross_entropy(output, target).sum().item()
            pred = (output >= 0.5).long()
            correct = pred.eq(target.view_as(pred)).sum().item()

            # logs
            monitor.update("test loss", test_loss)
            monitor.update("accuracy", correct / data.shape[0])
            stream.set_description(
                "Epoch: {epoch}. {monitor}".format(epoch=epoch, monitor=monitor)
            )
    return monitor

In [42]:
def train_test(model, optimizer, train_loader, test_loader, device, n_epochs=1):
    for epoch in range(0, n_epochs):
        train(model, train_loader, optimizer, epoch, device)
        test(model, test_loader, epoch, device)

In [43]:
train_test(model=model,
           optimizer=optimizer,
           train_loader=train_dataloader,
           test_loader=test_dataloader,
           device="cpu",
           n_epochs=70
           )

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

...

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

KeyboardInterrupt: 

## transformers

In [ ]:
%pip install -q evaluate
%pip install accelerate -U -q

In [11]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer
from datasets import Dataset
import evaluate

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [57]:
X = df['Description Cleaned Translated']
y = df['Analysis results']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [58]:
accuracy = evaluate.load("accuracy")

id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [59]:
class RacismDatasetHF(Dataset):
    def __init__(self, x, y):
        self.seq_tweets = x
        self.labels = y

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        tweet = self.seq_tweets.iloc[idx]
        label = self.labels.iloc[idx]

        return {'label': label, 'text': tweet}

In [60]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [61]:
# weird way to convert pytorch dataset into huggingface dataset

torch_dataset_train = RacismDatasetHF(X_train, y_train)
torch_dataset_test = RacismDatasetHF(X_test, y_test)


def gen():
    for idx in range(len(torch_dataset_train)):
        yield torch_dataset_train[idx]

def gen2():
    for idx in range(len(torch_dataset_test)):
        yield torch_dataset_test[idx]


train_dset = Dataset.from_generator(gen)
test_dset = Dataset.from_generator(gen2)

hf_train = train_dset.map(preprocess_function, batched=True)
hf_test = test_dset.map(preprocess_function)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [62]:
training_args = TrainingArguments(
    output_dir="distil_vanilla",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_train,
    eval_dataset=hf_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [63]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.552905,0.720000
2,No log,0.539752,0.745000
3,No log,0.498665,0.767500
4,No log,0.501859,0.790000
5,No log,0.567394,0.787500
6,No log,0.620721,0.802500
7,No log,0.674715,0.797500
8,No log,0.693468,0.795000
9,No log,0.739885,0.797500
10,No log,0.768870,0.810000


Checkpoint destination directory distil_vanilla/checkpoint-25 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory distil_vanilla/checkpoint-50 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory distil_vanilla/checkpoint-75 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory distil_vanilla/checkpoint-100 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory distil_vanilla/checkpoint-125 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory distil_vanilla/checkpoint-150 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory distil_vanilla/checkpoint-175 already exists and is non-empty. Saving 

TrainOutput(global_step=500, training_loss=0.10959545135498047, metrics={'train_runtime': 307.601, 'train_samples_per_second': 104.031, 'train_steps_per_second': 1.625, 'total_flos': 651838951936512.0, 'train_loss': 0.10959545135498047, 'epoch': 20.0})

In [55]:
from transformers import pipeline

text = "adsf adsf"
classifier = pipeline("sentiment-analysis", model="philip1231/distil_low_lr")
classifier(text)

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

[{'label': 'NEGATIVE', 'score': 0.5450683832168579}]

In [64]:
# !huggingface-cli login

trainer.push_to_hub("no_processing")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1709389906.9c73e5f59602.1721.2:   0%|          | 0.00/11.7k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

events.out.tfevents.1709391345.9c73e5f59602.1721.3:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/philip1231/distil_vanilla/commit/000101d3079813f30cb52ff6b46e8710a4c9ae04', commit_message='no_processing', commit_description='', oid='000101d3079813f30cb52ff6b46e8710a4c9ae04', pr_url=None, pr_revision=None, pr_num=None)